In [ ]:
import datetime as dt
from calendar import monthrange
import s3fs
import xarray as xr
import pandas as pd
import numpy as np
import os

In [ ]:
def get_water_data(sat: str, lyr: int, idyjl: int, band: int = 9):
    """
    Gets GOES water vapor data from AWS S3.

    Args:
        sat: 'goes-east' (GOES-16), 'goes-west' (GOES-17/18)
        yr: Year (2025)
        ldyjl: Julian day (1-365)
        band: ABI band (8=upper-level, 9=mid-level, 10=lower-level)
    
    Return:
        xarray.Dataset of the selected day's water vapor imagery.
    """

    #converts julian data to date-time
    d = dt.datetime(lyr,1,1) + dt.timedelta(days=idyjl - 1)
    #connects to s3 bucket
    fs = s3fs.S3FileSystem(anon=True)
    
    imon,idym=d.month,d.day
    syr,sjdy,smon,sdym = str(lyr).zfill(4),str(idyjl).zfill(3),str(imon).zfill(2),str(idym).zfill(2)
    
    # Choose bucket
    sat_bucket = 'noaa-goes16' if sat == 'goes-east' else 'noaa-goes17'
    sat_bucket = 'noaa-goes16' if sat == 'goes-east' else "noaa-goes19"

    # Path to GOES ABI L2 CMIP (Cloud & Moisture Imagery)
    path_pattern = f"s3://{sat_bucket}/ABI-L2-CMIPF/{syr}/{sjdy}/*/OR_ABI-L2-CMIPF-M6C{str(band).zfill(2)}*.nc"

    # List files and limit
    file_location = fs.glob(path_pattern)[:3]
    var = 'CMI'
    if not file_location:
        raise FileNotFoundError(f"No files found for pattern: {path_pattern}")

    file_ob = [fs.open(file) for file in file_location]        #open connection to files
    
    #open all the day's data
    ds = xr.open_mfdataset(file_ob,combine='nested',concat_dim='time') #note file is super messed up formatting

    ds['BT'] = ds[var]
    ds['BT'].attrs['units'] = 'Kelvin'
    ds['BT'].attrs['long_name'] = 'Brightness Temperature / CMI'
    
    return ds

In [ ]:
regions = {
    "Western Hemisphere": {"x_min": 0, "x_max": 2700, "y_min": 0, "y_max": 5424},
    "Tropics": {"x_min": 1000, "x_max": 4400, "y_min": 2000, "y_max": 3400},
    "North America": {"x_min": 1000, "x_max": 2200, "y_min": 3000, "y_max": 4400},
    "South America": {"x_min": 1200, "x_max": 2600, "y_min": 1000, "y_max": 3000}
}

# Choose satellite automatically
def get_satellite_for_date(date):
    """GOES-16 through April 7 2025 (day 97), then GOES-19."""
    return 'goes-east' if date.timetuple().tm_yday <= 97 else 'goes-19'

daily_csv = "goes16_water_vapor_regions_daily_2025.csv"
daily_data = []

# If CSV already exists, load it to continue appending
try:
    df_existing = pd.read_csv(daily_csv, parse_dates=["date"])
    last_date = df_existing["date"].max().date()
    start_day = (last_date - dt.date(2025, 1, 1)).days + 2
    daily_data = df_existing.to_dict("records")
    print(f"Starting from {last_date + dt.timedelta(days=1)} (day {start_day})")
except FileNotFoundError:
    start_day = 1

# Collect daily data
for day_of_year in range(start_day, 366):
    date = dt.datetime(2025, 1, 1) + dt.timedelta(days=day_of_year - 1)
    satellite = get_satellite_for_date(date)

    try:
        ds = get_water_data(satellite, 2025, day_of_year, band=9)
    except FileNotFoundError:
        print(f"No data for {date.date()} ({satellite}) — skipping")
        continue

    for region_name, bbox in regions.items():
        region_ds = ds.isel(
            x=slice(bbox["x_min"], bbox["x_max"]),
            y=slice(bbox["y_min"], bbox["y_max"])
        )

        mean_bt = float(region_ds["BT"].mean())
        min_bt = float(region_ds["BT"].min())
        max_bt = float(region_ds["BT"].max())

        record = {
            "date": date.date(),
            "region": region_name,
            "mean_BT": mean_bt,
            "min_BT": min_bt,
            "max_BT": max_bt
        }
        daily_data.append(record)

        #pd.DataFrame(daily_data).to_csv(daily_csv, index=False)

# Make weekly summary
df = pd.DataFrame(daily_data)
df["date"] = pd.to_datetime(df["date"])
df["week"] = df["date"].dt.isocalendar().week

weekly_df = (
    df.groupby(["week", "region"])
      .agg({
          "mean_BT": "mean",
          "min_BT": "min",
          "max_BT": "max"
      })
      .reset_index()
)

weekly_df["week_start_date"] = (
    pd.to_datetime("2025-01-01") + pd.to_timedelta((weekly_df["week"] - 1) * 7, unit="d")
)

#weekly_df.to_csv("goes16_water_vapor_regions_weekly_2025.csv", index=False)

✅ Resuming from 2025-04-06 (day 96)


/var/folders/j_/fn1ljbfd0d30pzhjdjyjldz80000gn/T/ipykernel_3849/1882077771.py:43: UserWarning: rename 't' to 'time' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  ds = ds.rename({'t': 'time'})


{'date': datetime.date(2025, 4, 6), 'region': 'Western Hemisphere', 'mean_BT': 246.24407958984375, 'min_BT': 185.03512573242188, 'max_BT': 269.4169616699219}
{'date': datetime.date(2025, 4, 6), 'region': 'Tropics', 'mean_BT': 243.6876220703125, 'min_BT': 185.03512573242188, 'max_BT': 262.6427001953125}
{'date': datetime.date(2025, 4, 6), 'region': 'North America', 'mean_BT': 253.39512634277344, 'min_BT': 215.05355834960938, 'max_BT': 267.5540466308594}
{'date': datetime.date(2025, 4, 6), 'region': 'South America', 'mean_BT': 250.75802612304688, 'min_BT': 185.03512573242188, 'max_BT': 268.65484619140625}


/var/folders/j_/fn1ljbfd0d30pzhjdjyjldz80000gn/T/ipykernel_3849/1882077771.py:43: UserWarning: rename 't' to 'time' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  ds = ds.rename({'t': 'time'})


{'date': datetime.date(2025, 4, 7), 'region': 'Western Hemisphere', 'mean_BT': 245.85089111328125, 'min_BT': 187.23675537109375, 'max_BT': 269.5863037109375}
{'date': datetime.date(2025, 4, 7), 'region': 'Tropics', 'mean_BT': 243.0523223876953, 'min_BT': 183.42623901367188, 'max_BT': 265.18304443359375}
{'date': datetime.date(2025, 4, 7), 'region': 'North America', 'mean_BT': 252.25331115722656, 'min_BT': 212.8095703125, 'max_BT': 265.3947448730469}
{'date': datetime.date(2025, 4, 7), 'region': 'South America', 'mean_BT': 249.72633361816406, 'min_BT': 189.81944274902344, 'max_BT': 268.90887451171875}


...

{'date': datetime.date(2025, 11, 8), 'region': 'Western Hemisphere', 'mean_BT': 247.15740966796875, 'min_BT': 186.26295471191406, 'max_BT': 266.3262023925781}
{'date': datetime.date(2025, 11, 8), 'region': 'Tropics', 'mean_BT': 248.38475036621094, 'min_BT': 185.54318237304688, 'max_BT': 263.4048156738281}
{'date': datetime.date(2025, 11, 8), 'region': 'North America', 'mean_BT': 255.99591064453125, 'min_BT': 218.86407470703125, 'max_BT': 265.31005859375}
{'date': datetime.date(2025, 11, 8), 'region': 'South America', 'mean_BT': 250.45388793945312, 'min_BT': 186.26295471191406, 'max_BT': 266.3262023925781}


/var/folders/j_/fn1ljbfd0d30pzhjdjyjldz80000gn/T/ipykernel_3849/1882077771.py:43: UserWarning: rename 't' to 'time' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  ds = ds.rename({'t': 'time'})


{'date': datetime.date(2025, 11, 9), 'region': 'Western Hemisphere', 'mean_BT': 247.19900512695312, 'min_BT': 191.34364318847656, 'max_BT': 265.90283203125}
{'date': datetime.date(2025, 11, 9), 'region': 'Tropics', 'mean_BT': 249.66464233398438, 'min_BT': 185.54318237304688, 'max_BT': 264.12457275390625}
{'date': datetime.date(2025, 11, 9), 'region': 'North America', 'mean_BT': 255.58966064453125, 'min_BT': 219.96487426757812, 'max_BT': 264.92901611328125}
{'date': datetime.date(2025, 11, 9), 'region': 'South America', 'mean_BT': 252.70172119140625, 'min_BT': 191.34364318847656, 'max_BT': 265.90283203125}


/var/folders/j_/fn1ljbfd0d30pzhjdjyjldz80000gn/T/ipykernel_3849/1882077771.py:43: UserWarning: rename 't' to 'time' does not create an index anymore. Try using swap_dims instead or use set_index after rename to create an indexed coordinate.
  ds = ds.rename({'t': 'time'})


{'date': datetime.date(2025, 11, 10), 'region': 'Western Hemisphere', 'mean_BT': 246.0333251953125, 'min_BT': 184.7810821533203, 'max_BT': 265.22540283203125}
{'date': datetime.date(2025, 11, 10), 'region': 'Tropics', 'mean_BT': 247.73988342285156, 'min_BT': 183.21453857421875, 'max_BT': 261.8382568359375}
{'date': datetime.date(2025, 11, 10), 'region': 'North America', 'mean_BT': 254.45394897460938, 'min_BT': 217.67857360839844, 'max_BT': 265.22540283203125}
{'date': datetime.date(2025, 11, 10), 'region': 'South America', 'mean_BT': 249.93389892578125, 'min_BT': 184.7810821533203, 'max_BT': 261.287841796875}
⚠️ No data for 2025-11-11 (goes-19) — skipping
⚠️ No data for 2025-11-12 (goes-19) — skipping
⚠️ No data for 2025-11-13 (goes-19) — skipping
⚠️ No data for 2025-11-14 (goes-19) — skipping
⚠️ No data for 2025-11-15 (goes-19) — skipping
⚠️ No data for 2025-11-16 (goes-19) — skipping
⚠️ No data for 2025-11-17 (goes-19) — skipping
⚠️ No data for 2025-11-18 (goes-19) — skipping
⚠️ No d